In [16]:
import numpy as np
import pandas as pd
import neptune
import plotly.express as px
import plotly.graph_objects as go
from math import log

In [23]:
from typing import List


project = neptune.init_project(
  project="pmtest/llm-random",
  mode="read-only",
  api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIyMDY0ZDI5Ni05YWU3LTQyNGYtYmY4My1hZTFkY2EzYmUwMjgifQ==",
)
columns = [
    "sys/tags",
    "loss_interval/100",
    "args/learning_rate",
    "sys/name",
    'args/grad_modif_params',
]

def infere_layer_type(tags: str):
    tags = tags.split(',')
    if "true_baseline" in tags:
        return "baseline"
    else:
        return "regular"

def infere_c(grad_modif_params: str):
    params = grad_modif_params.split(',')
    for p in params:
        key, val = p.split('=')
        if key == "c":
            return float(val)
    return None

table = project.fetch_runs_table(tag="post_add_c_lr_grid", columns=columns).to_pandas()
table['layer_type'] = table['sys/tags'].apply(infere_layer_type)
table['c'] = table['args/grad_modif_params'].apply(infere_c)
table.rename(columns={'loss_interval/100': 'loss', 'args/learning_rate': 'lr', 'sys/id': 'id'}, inplace=True)


https://app.neptune.ai/pmtest/llm-random/


In [24]:
baseline = table[table['layer_type'] == 'baseline'].sort_values(by='lr')
rest = table[table['layer_type'] != 'baseline'].sort_values(by=['lr', 'c'])

fig = px.line(rest, x="lr", y=f"loss", title=f"c vs loss", log_x=True, log_y=True, color='c', markers=True, range_y=[4, 10])

fig.add_trace(go.Scatter(x=baseline['lr'], y=baseline['loss'], name='baseline',
                         line=dict(color='black', width=2, dash='dash')))

fig.update_layout(
    title=f"Loss vs Learning Rate for All Layer Typles and Placements",
    yaxis = dict(
        showexponent = 'all',
        exponentformat = 'power'
    ),
    xaxis = dict(
        showexponent = 'all',
        exponentformat = 'power'
    )
)
fig.show()